In [1]:
from stinespring_t_update_classes import stinespring_unitary_update as stinespring
from Stinespring_unitary_circuits import generate_gate_connections
import numpy as np
from q_lab_toolbox.settings import DecaySettings
from q_lab_toolbox.hamiltonians import decay_hamiltonian
from q_lab_toolbox.jump_operators import create_jump_operators

In [2]:
from decay.settings import settings as s

In [3]:
def misc():
    # Set initial dictionary with arguments
    par_dict = {
        "qubit_structure": s.qubit_structure,
        "steadystate_weight": s.steadystate_weight,
    }
    if s.circuit_type == "pulse based":
        par_dict.update(
            {
                "driving_H_interaction": s.circuit_settings.driving_H_interaction,
                "control_H": s.circuit_settings.control_H,
                "T_pulse": s.circuit_settings.T_pulse,
                "lambdapar": s.circuit_settings.lambdapar,
                "Zdt": s.circuit_settings.Zdt,
            }
        )
    else:
        par_dict.update(
            {
                "n_grad_directions": s.circuit_settings.n_grad_directions,
                "cutoff": s.circuit_settings.cutoff,
            }
        )

    # Set entangle gate dictionary
    entangle_pars = {
        "t_ryd": s.circuit_settings.t_ryd,
        "phi": s.circuit_settings.phi,
        "gammat": s.circuit_settings.gammat,
    }

    # Create correct parameter array that will be optimized
    theta0 = (
        np.ones([s.circuit_settings.depth, 2 * s.m + 1, 3]) * np.pi / 2
    )  # Initial theta guess
    pars_per_layer = len(
        generate_gate_connections(2 * s.m + 1, structure=s.qubit_structure, cutoff=True)
    )
    gate_par = 0
    if s.circuit_type == "xy":
        phi0 = (
            np.ones([s.circuit_settings.depth, pars_per_layer]) * s.circuit_settings.phi
        )
        gate_par = phi0
        theta0 = np.concatenate((np.ravel(theta0), np.ravel(phi0)))
    elif s.circuit_type == "ryd":
        t_ryd0 = (
            np.ones(
                [
                    s.circuit_settings.depth,
                ]
            )
            * s.circuit_settings.t_ryd
        )
        gate_par = t_ryd0
        theta0 = np.concatenate((np.ravel(theta0), np.ravel(t_ryd0)))
    elif s.circuit_type == "decay":
        gammat0 = (
            np.ones([s.circuit_settings.depth, pars_per_layer])
            * s.circuit_settings.gammat
        )
        gate_par = s.circuit_settings.gammat0
        theta0 = np.concatenate((np.ravel(theta0), np.ravel(gammat0)))
    elif s.circuit_type == "pulse based":
        if "+11" in s.circuit_settings.control_H:
            n_controls_H = 2 * (2 * s.m + 1)
        else:
            n_controls_H = 2 * s.m + 1
        theta0 = np.zeros((n_controls_H, s.circuit_settings.Zdt, 2)) + 0.02
        # =============================================================================
        #     theta0[0,:,0] = np.sin(np.linspace(0,T_pulse, Zdt) + np.random.rand()*2*np.pi) *0.1*np.random.rand()
        #     theta0[0,:,1] = np.sin(np.linspace(0,T_pulse, Zdt) + np.random.rand()*2*np.pi) *0.1*np.random.rand()
        #     theta0[1,:,0] = np.sin(np.linspace(0,T_pulse, Zdt) + np.random.rand()*2*np.pi) *0.1*np.random.rand()
        #     theta0[1,:,1] = np.sin(np.linspace(0,T_pulse, Zdt) + np.random.rand()*2*np.pi) *0.1*np.random.rand()
        #     theta0[3,:,0] = np.sin(np.linspace(0,T_pulse, Zdt) + np.random.rand()*2*np.pi) *0.1*np.random.rand()
        #     theta0[3,:,1] = np.sin(np.linspace(0,T_pulse, Zdt) + np.random.rand()*2*np.pi) *0.1*np.random.rand()
        # =============================================================================
        theta0 = np.ravel(theta0)
    else:
        theta0 = np.ravel(theta0)

    # Set parameter dictionaries
    train_par = {
        "n_training": s.n_training,
        "seed": s.seed,
        "depth": s.circuit_settings.depth,
        "theta0": theta0,
        "max_it_training": s.max_it_training,
        "epsilon": s.epsilon,
        "gamma": s.gamma,
        "sigmastart": s.sigmastart,
        "circuit_type": s.circuit_type,
        "pauli_type": s.pauli_type,
        "t_repeated": s.t_repeated,
    }

    return train_par, par_dict, entangle_pars


train_par, par_dict, entangle_pars = misc()

In [4]:
stine = stinespring(par_dict=par_dict)

In [5]:
target = DecaySettings(
    ryd_interaction=0.2, m=2, omegas=(0.3, 0.5), gammas=(0.2, 0.4)
)

H = decay_hamiltonian(target).full()

An = [operator.full() for operator in create_jump_operators(target)]
stine.set_original_lindblad(H, An, 0.1)

In [6]:
from q_lab_toolbox.unitary_circuits import HardwareAnsatz

In [7]:
ha = HardwareAnsatz(m=2, n_qubits=5, depth=5)

In [8]:
phi = ha.phi_prime(ha.init_theta())

In [9]:
import qutip as qt

state00 = qt.Qobj([[1, 0], [0, 0]])
ancilla = qt.tensor( [state00 for _ in range(3)] )

random_ket = qt.rand_ket_haar(dims=[[2] * 2, [1] * 2], seed=5)
random_bra = random_ket.dag()

rho = random_ket * random_bra

random_ket2 = qt.rand_ket_haar(dims=[[2] * 3, [1] * 3], seed=5)
random_bra2 = random_ket2.dag()

rho2 = random_ket2 * random_bra2

In [10]:
U = phi(rho)

In [11]:
rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.04523462+0.j          0.02854258+0.01241458j -0.01584032-0.17853504j
   0.0303299 -0.09578135j]
 [ 0.02854258-0.01241458j  0.02141724+0.j         -0.05899377-0.10830645j
  -0.00714919-0.06876105j]
 [-0.01584032+0.17853504j -0.05899377+0.10830645j  0.71020114+0.j
   0.36741531+0.15324894j]
 [ 0.0303299 +0.09578135j -0.00714919+0.06876105j  0.36741531-0.15324894j
   0.223147  +0.j        ]]

In [12]:
from training_data import mk_training_data2
from q_lab_toolbox.qubit_readout_operators import create_readout_computational_basis

Os = create_readout_computational_basis(target)

data = mk_training_data2(rho, 0.1, 3, Os, target)

In [13]:
ha.J(ha.init_theta(), training_data=data)

0.6369191708673749